In [1]:
import os
import pandas as pd
import wget
import sys
import subprocess
from subprocess import call
import matplotlib.pyplot as plt
import seaborn 
import random
%matplotlib inline

import json

import numpy as np

from metamind.api import ClassificationData, ClassificationModel, set_api_key

sys.path.append(os.environ["HOME"])
from ds_config import *

print os.environ["HOME"]
MAIN_PATH = os.environ["HOME"]+"/GLaDOS_Workspace/"
os.mkdir(MAIN_PATH)
os.mkdir(MAIN_PATH+"Videos/")
os.mkdir(MAIN_PATH+"Images/")

seaborn.set_style("white")

/Users/williamspearman


# FFMPEG Scrips

In [2]:
def getVideoInfo(file_name):
    bashCommand = "ffprobe -show_streams " + file_name
    process = subprocess.Popen(bashCommand.split(" "), stdout=subprocess.PIPE)
    output = process.communicate()[0].split("\n")
    ret_dict = { }
    for x in output:
        if "=" in x:
            key_vals = x.split("=")
            ret_dict[key_vals[0]] = key_vals[1]
    #print ret_dict
    return ret_dict

def getRandInt(int1, int2):
    if int1[0]>=int1[1]:
        return np.random.randint(int2[0],int2[1])
    elif int2[0]>=int2[1]:
        return np.random.randint(int1[0],int1[1])
    
    if np.random.random() > 0.5:
        return np.random.randint(int1[0],int1[1])
    else:
        return np.random.randint(int2[0],int2[1])

    

def getClipTriplets(file_name, snap_time, pm_secs=1.):
    ret_val = { }
    info_dict = getVideoInfo(file_name)
    required_vals = { "duration", "nb_frames", "avg_frame_rate", "sample_aspect_ratio" }
    if len(required_vals.intersection(set(info_dict.keys())))!=len(required_vals):
        return None
    frame_rate = 0.
    try:
        frame_rate = float(info_dict["avg_frame_rate"].split("/")[0])/float(info_dict["avg_frame_rate"].split("/")[1])
    except:
        return None
    num_frames = float(info_dict["nb_frames"])
    duration = float(info_dict["duration"])

    if int(num_frames/frame_rate-duration)!= 0:
        return None
    
    pm_frames = pm_secs * frame_rate
    snap_frame = snap_time * frame_rate
    
    snap_frame_low = 0.
    snap_frame_med = snap_frame_low+1.*pm_frames
    snap_frame_high = snap_frame_low+2.*pm_frames
    while snap_frame_high<=num_frames:
        ret_val["TIME_%i_%i" % (snap_frame_low, snap_frame_high)] = \
        [ snap_frame_low/frame_rate, snap_frame_med/frame_rate, snap_frame_high/frame_rate ]
        snap_frame_low += pm_frames
        snap_frame_med += pm_frames
        snap_frame_high += pm_frames
    
    
    return ret_val
    
def downloadUrl(url):
    base_name = ""
    try:
        base_name = url.split("/")[-1]
    except:
        return ""
    
    output_name = MAIN_PATH + "Videos/" + base_name
    if os.path.isfile(output_name)==False:
        wget.download(url, MAIN_PATH+"Videos/")
    
    if os.path.isfile(output_name)==True:
        return output_name
    else:
        return ""


def getGetSignalBkgTriplets(file_name, snap_time, pm_secs=1.):
    ret_val = { }
    info_dict = getVideoInfo(file_name)
    required_vals = { "duration", "nb_frames", "avg_frame_rate", "sample_aspect_ratio" }
    if len(required_vals.intersection(set(info_dict.keys())))!=len(required_vals):
        return None
    frame_rate = 0.
    try:
        frame_rate = float(info_dict["avg_frame_rate"].split("/")[0])/float(info_dict["avg_frame_rate"].split("/")[1])
    except:
        return None
    num_frames = float(info_dict["nb_frames"])
    duration = float(info_dict["duration"])

    if int(num_frames/frame_rate-duration)!= 0:
        return None
    
    pm_frames = pm_secs * frame_rate
    snap_frame = snap_time * frame_rate
    
    snap_frame_low = snap_frame-pm_frames
    snap_frame_high = snap_frame+pm_frames
    if snap_frame_low<0:
        return None
    if snap_frame_high>num_frames:
        return None
    
    int1 = [ pm_frames, max(0,snap_frame_low-pm_frames) ]
    int2 = [ min(pm_frames+snap_frame_high, num_frames), num_frames-pm_frames ]
    bkg_frame = getRandInt(int1, int2)
    
    ret_val["SNAP"] = [ snap_frame_low/frame_rate, snap_frame/frame_rate, snap_frame_high/frame_rate ]
    ret_val["NOT_SNAP"] = [ (bkg_frame-pm_frames)/frame_rate, bkg_frame/frame_rate, (bkg_frame+pm_frames)/frame_rate ]
    
    return ret_val
    
def downloadUrl(url):
    base_name = ""
    try:
        base_name = url.split("/")[-1]
    except:
        return ""
    
    output_name = MAIN_PATH + "Videos/" + base_name
    if os.path.isfile(output_name)==False:
        wget.download(url, MAIN_PATH+"Videos/")
    
    if os.path.isfile(output_name)==True:
        return output_name
    else:
        return ""



def makeImageMontage(url, snap_time, time_offset=0.5, triplets=True):
    ret_val = [ ]
    video_path = downloadUrl(url)
    #print "BLah %f" % 0.2
    base_name = ""
    try:
        base_name = video_path.split("/")[-1].split(".")[0]
    except:
        print "Failed splitting: %s" % video_path
        return None
    input_times = None
    if triplets:
        input_times = getGetSignalBkgTriplets(video_path, snap_time, time_offset)
    else:
        input_times = getClipTriplets(video_path, snap_time, time_offset)
    if input_times==None:
        return ret_val
    #resolution = resolution.replace(":", "x")
    #name1 = "blah"
    #name2 = "blah"
    ffmpeg_cmds = { }
    file_names = { }
    for mykey in input_times.keys():
        output_name = MAIN_PATH + "Images/" + base_name
        ffmpeg_cmds[mykey] = [ ]
        file_names[mykey] = [ ]
        for x, i in zip(input_times[mykey], range(0,len(input_times[mykey]))):
            file_name = "%s_%i_%s.jpg" % (output_name, i, mykey)
            file_names[mykey].append(file_name)
            ffmpeg_cmds[mykey].append("ffmpeg -ss %f -i %s -vframes 1 %s 2> /dev/null" % (x, video_path, file_name))
        #montage_cmd = "montage -tile 1x3 -geometry 512x256+0+0 "
        montage_cmd = "montage -tile 1x3 -geometry 512x+0+0 "
        for thecmd, fname in zip(ffmpeg_cmds[mykey], file_names[mykey]):
            call(thecmd, shell=True)
            montage_cmd = montage_cmd+fname+" "
        montage_name = "%s_%i_%s_MONTAGE.jpg" % (output_name, int(1000*input_times[mykey][1]), mykey)
        montage_cmd = montage_cmd + " " + montage_name
        call(montage_cmd,shell=True)
        for fname in file_names[mykey]:
            try:
                os.remove(fname)
            except:
                print "Couldn't remove %s" % fname
        ret_val.append((montage_name, mykey))
        
        
        
            
    return ret_val


In [91]:
set_api_key(metamind_api_key)
classifier = ClassificationModel(id=18410)

Hello, Ben Cook


In [93]:
#singleClips = makeImageMontage("http://vf.hudl.com/9n6/81k/bab387d7-c875-4df6-89f7-3a0c910deb6a/lelyp36kuaykgm1j-3000_1100.mp4", 8.809, 0.5, False)

def DetectSnap(url, snap_time, time_offset=1.):
    base_name = url.split("/")[-1].split(".mp4")[0]
    singleClips = makeImageMontage(url, snap_time, time_offset, False)
    classifier = ClassificationModel(id=18410)

    predictions = [ ]

    length = len(singleClips)
    if length<=0:
        return
    for x, i in zip(singleClips, range(0,length)):
        print "On %i of %i" % (i, length)
        #time = float(x[1].split("_")[-1])
        time = (float(x[1].split("_")[-1])+float(x[1].split("_")[-2]))/2.
        result = classifier.predict(x[0], input_type="files")
        prediction = result[0]['label']
        prob_snap=0.
        if prediction=="SNAP":
            prob_snap=result[0]["probability"]
        else:
            prob_snap=1-result[0]["probability"]
        print "Probability %f for time %f" % (prob_snap, time)
        predictions.append( (time, prob_snap) )
        
    predictions = sorted(predictions)
    np_arr = np.array(predictions)

    x = [ ]

    for tmp in np_arr[:,0]:
        x.append(tmp/30.)
    y = np_arr[:,1]
    #print x
    #print y

    #x2 = [ 8.809 ]
    #y2 = [ 1. ]

    plt.plot(x,y)
    plt.axvline(snap_time, color='r', linestyle='dashed', linewidth=2) 
    plt.xlabel('Seconds', fontsize=12)
    plt.ylabel('Probability of Snap', fontsize=12)
    #plt.show()
    plt.savefig("SnapProbability_%s.png" % base_name)
    plt.show()
    return np_arr

In [75]:
formationRDD = sc.textFile("s3n://hudl-hadoop/GLaDOS/formation_data/").map(json.loads)

In [78]:
files_to_run = formationRDD.map(lambda x: (x["video"]["snap time"], x["video"]["urls"][0])).takeSample(False,10)
random.shuffle(files_to_run)

In [79]:
data = [ ]

for x in files_to_run:
    data.append(DetectSnap(x[1],x[0],0.5))

On 0 of 24


Traceback (most recent call last):
  File "/Users/williamspearman/anaconda/lib/python2.7/site-packages/metamind/api/utils.py", line 33, in make_request
    return validate_result(response)
  File "/Users/williamspearman/anaconda/lib/python2.7/site-packages/metamind/api/utils.py", line 119, in validate_result
    raise ValueError("Unknown server error (error code: %d, url: %s)" % (response.status_code, response.url))
ValueError: Unknown server error (error code: 504, url: https://api.metamind.io/api/v1.2/classifiers/18410/predict)


KeyboardInterrupt: 